In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score
import pickle

In [2]:
dirloc = 'train_test_dataset'
train_x = pd.read_csv('{}/train_x.csv'.format(dirloc))
train_y = pd.read_csv('{}/train_y.csv'.format(dirloc))
valid_x = pd.read_csv('{}/valid_x.csv'.format(dirloc))
valid_y = pd.read_csv('{}/valid_y.csv'.format(dirloc))
test_x = pd.read_csv('{}/test_x.csv'.format(dirloc))
test_y = pd.read_csv('{}/test_y.csv'.format(dirloc))

In [3]:
train_x

,0,1,2,3,4,5,6,7,8,9,...,2429,2430,2431,2432,2433,2434,2435,2436,2437,2438
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Columns: 2439 entries, 0 to 2438
dtypes: int64(2439)
memory usage: 14.9 MB


In [5]:
train_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   CLASS   800 non-null    int64
dtypes: int64(1)
memory usage: 6.4 KB


In [6]:
valid_x

,0,1,2,3,4,5,6,7,8,9,...,2429,2430,2431,2432,2433,2434,2435,2436,2437,2438
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
hp_space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform('gamma', 1,9),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimator' : 180
    }

In [8]:
def objective(space):
        clf = xgb.XGBClassifier(
        objective = 'binary:logistic',
        missing = 1,
        n_estimator = hp_space['n_estimator'],
        max_depth = hp_space['max_depth'],
        gamma = hp_space['gamma'],
        min_child_weight = hp_space['min_child_weight'],
    )
        evaluation = [(train_x, train_y), ( valid_x, valid_y)]
            
        clf.fit(train_x,train_y,eval_set = evaluation,verbose=True,early_stopping_rounds=10,eval_metric='aucpr')
        
        pred_y = clf.predict(test_x)
        accuracy = accuracy_score(test_y, pred_y>0.5)
        print ("SCORE:", accuracy)
        return {'loss': -accuracy, 'status': STATUS_OK ,'Trained_model': clf}

In [13]:
def getBestModelfromTrials(trials):
    valid_trial_list = [trial for trial in trials
                            if STATUS_OK == trial['result']['status']]
    losses = [ float(trial['result']['loss']) for trial in valid_trial_list]
    index_having_minumum_loss = np.argmin(losses)
    best_trial_obj = valid_trial_list[index_having_minumum_loss]
    return best_trial_obj['result']['Trained_model']

In [10]:
trials = Trials()

In [11]:
best_hyperparams = fmin(fn = objective,
                        space = hp_space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

[13:12:03] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-aucpr:0.96757	validation_1-aucpr:0.95349
[1]	validation_0-aucpr:0.98159	validation_1-aucpr:0.97665
[2]	validation_0-aucpr:0.98366	validation_1-aucpr:0.98154
[3]	validation_0-aucpr:0.98621	validation_1-aucpr:0.98239
[4]	validation_0-aucpr:0.98672	validation_1-aucpr:0.98270
[5]	validation_0-aucpr:0.98679	validation_1-aucpr:0.98298
[6]	validation_0-aucpr:0.98880	validation_1-aucpr:0.98736
[7]	validation_0-aucpr:0.98883	validation_1-aucpr:0.98685
[8]	validation_0-aucpr:0.99024	validation_1-aucpr:0.99016
[9]	validation_0-aucpr:0.99085	validation

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

[0]	validation_0-aucpr:0.96757	validation_1-aucpr:0.95349                         
[1]	validation_0-aucpr:0.98159	validation_1-aucpr:0.97665                         
[2]	validation_0-aucpr:0.98366	validation_1-aucpr:0.98154                         
[3]	validation_0-aucpr:0.98621	validation_1-aucpr:0.98239                         
[4]	validation_0-aucpr:0.98672	validation_1-aucpr:0.98270                         
[5]	validation_0-aucpr:0.98679	validation_1-aucpr:0.98298                         
[6]	validation_0-aucpr:0.98880	validation_1-aucpr:0.98736                         
[7]	validation_0-aucpr:0.98883	validation_1-aucpr:0.98685                         
[8]	validation_0-aucpr:0.99024	validation_1-aucpr:0.99016                         
[9]	validation_0-aucpr:0.99085	validation_1-aucpr:0.99049                         
[10]	validation_0-aucpr:0.99091	validation_1-aucpr:0.99091                        
[11]	validation_0-aucpr:0.99184	validation_1-aucpr:0.99112                        
[12]

...

[0]	validation_0-aucpr:0.96757	validation_1-aucpr:0.95349                          
[1]	validation_0-aucpr:0.98159	validation_1-aucpr:0.97665                          
[2]	validation_0-aucpr:0.98366	validation_1-aucpr:0.98154                          
[3]	validation_0-aucpr:0.98621	validation_1-aucpr:0.98239                          
[4]	validation_0-aucpr:0.98672	validation_1-aucpr:0.98270                          
[5]	validation_0-aucpr:0.98679	validation_1-aucpr:0.98298                          
[6]	validation_0-aucpr:0.98880	validation_1-aucpr:0.98736                          
[7]	validation_0-aucpr:0.98883	validation_1-aucpr:0.98685                          
[8]	validation_0-aucpr:0.99024	validation_1-aucpr:0.99016                          
[9]	validation_0-aucpr:0.99085	validation_1-aucpr:0.99049                          
[10]	validation_0-aucpr:0.99091	validation_1-aucpr:0.99091                         
[11]	validation_0-aucpr:0.99184	validation_1-aucpr:0.99112                  

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

[0]	validation_0-aucpr:0.96757	validation_1-aucpr:0.95349                          
[1]	validation_0-aucpr:0.98159	validation_1-aucpr:0.97665                          
[2]	validation_0-aucpr:0.98366	validation_1-aucpr:0.98154                          
[3]	validation_0-aucpr:0.98621	validation_1-aucpr:0.98239                          
[4]	validation_0-aucpr:0.98672	validation_1-aucpr:0.98270                          
[5]	validation_0-aucpr:0.98679	validation_1-aucpr:0.98298                          
[6]	validation_0-aucpr:0.98880	validation_1-aucpr:0.98736                          
[7]	validation_0-aucpr:0.98883	validation_1-aucpr:0.98685                          
[8]	validation_0-aucpr:0.99024	validation_1-aucpr:0.99016                          
[9]	validation_0-aucpr:0.99085	validation_1-aucpr:0.99049                          
[10]	validation_0-aucpr:0.99091	validation_1-aucpr:0.99091                         
[11]	validation_0-aucpr:0.99184	validation_1-aucpr:0.99112                  

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

C:\Users\62819\anaconda3\envs\data-scientist\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

[0]	validation_0-aucpr:0.96757	validation_1-aucpr:0.95349                          
[1]	validation_0-aucpr:0.98159	validation_1-aucpr:0.97665                          
[2]	validation_0-aucpr:0.98366	validation_1-aucpr:0.98154                          
[3]	validation_0-aucpr:0.98621	validation_1-aucpr:0.98239                          
[4]	validation_0-aucpr:0.98672	validation_1-aucpr:0.98270                          
[5]	validation_0-aucpr:0.98679	validation_1-aucpr:0.98298                          
[6]	validation_0-aucpr:0.98880	validation_1-aucpr:0.98736                          
[7]	validation_0-aucpr:0.98883	validation_1-aucpr:0.98685                          
[8]	validation_0-aucpr:0.99024	validation_1-aucpr:0.99016                          
[9]	validation_0-aucpr:0.99085	validation_1-aucpr:0.99049                          
[10]	validation_0-aucpr:0.99091	validation_1-aucpr:0.99091                         
[11]	validation_0-aucpr:0.99184	validation_1-aucpr:0.99112                  

In [14]:
model = getBestModelfromTrials(trials)

In [18]:
pred_y = model.predict(test_x)

In [22]:
from sklearn.metrics import accuracy_score 
accuracy_score(test_y,pred_y)

0.9217391304347826

# save model

In [21]:
pickle.dump(model, open("model/xgboost_best_model.pkl", "wb"))